### Importando Bibliotecas

In [431]:
! python -m spacy download pt_core_news_sm &> /dev/null
! pip install yake &> /dev/null

In [432]:
import glob
import re
import os
import unicodedata
import math

import nltk
from nltk.corpus        import stopwords
from nltk.tokenize      import word_tokenize
from nltk.stem.snowball import SnowballStemmer

from tabulate import tabulate

import spacy
import pt_core_news_sm

import yake

nlp = spacy.load("pt_core_news_sm")
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Conectando ao Google Drive


In [433]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Configurações do algoritmo YAKE

In [434]:
language = "pt"
max_ngram_size = 1
deduplication_thresold = 0.9
deduplication_algo = 'seqm'
windowSize = 2
numOfKeywords = 20

### Importando os documentos



In [637]:
def read_file(directory: str) -> str:
    file = open(directory, 'r')
    text = file.read()

    if file.name == '/content/drive/MyDrive/Information retrieval/data/estrutura de dados [AV - 3](1 - GABARITO).txt':
        text = text.split('\n', 1)[1]
        text = re.sub(r"\bAlternativas\b[\s\S]*", "", text)

    return text

In [638]:
print(read_file('/content/drive/MyDrive/Information retrieval/data/estrutura de dados [AV - 2](2 - GABARITO).txt'))

Algoritmos e Estrutura de Dados - Arvores

Sobre árvores de pesquisa binária, analise as assertivas abaixo e assinale a alternativa correta.

I.   Admitem todas as operações sobre conjuntos dinâmicos, no pior caso, cada operação demora um tempo 1(n) em uma árvore com n elementos.
II.  As árvores rubro-negra são uma variante de árvores de pesquisa binária.
III. Em uma árvore de pesquisa binária construída aleatoriamente, não há como medir o tempo esperado para cada operação.
IV.  Uma árvore rubro-negra é uma árvore de pesquisa balanceada, chamada árvore B.

Alternativas

A. Todas as assertivas estão corretas. 
B. Todas as assertivas estão incorretas.
C. Apenas a assertiva I está correta.
D. Apenas as assertivas I e II estão corretas.
E. Apenas as assertivas III e IV estão corretas.


### Tratamento dos dados

##### Removendo acentuação

In [639]:
def remove_accents(text: str) -> str:
    nfkd = unicodedata.normalize('NFKD', text)

    filtered_text = ''.join([c for c in nfkd if not unicodedata.combining(c)])

    return filtered_text

In [640]:
print('Remoção de acentos: Análise ->', remove_accents('Análise'))

Remoção de acentos: Análise -> Analise


##### Removendo caracteres especiais

In [641]:
def remove_special_characters(text: str) -> str:
    regex = re.compile(r'[^a-zA-Z0-9\s]')

    filtered_text = re.sub(regex, '', text)

    return filtered_text

In [642]:
print('Remoção de caracteres especiais: Guarda-chuva ->', remove_special_characters('Guarda-chuva'))

Remoção de caracteres especiais: Guarda-chuva -> Guardachuva


##### Convertendo para minúsculas

In [643]:
def convert_to_lowercase(text: str) -> str:
    return text.casefold()

In [644]:
print('Conversão para minúsculas: Estrutura de Dados ->', convert_to_lowercase('Estrutura de Dados'))

Conversão para minúsculas: Estrutura de Dados -> estrutura de dados


##### Removendo *Stopwords*

In [645]:
def remove_stopwords(token: list[str]) -> list[str]:
    stop_words = stopwords.words('portuguese')

    filtered_token = [word for word in token if word not in stop_words]

    return filtered_token

In [646]:
print('Remoção de Stopwords: Todas aquelas pessoas -> ', remove_stopwords(['todas', 'aquelas', 'pessoas']))

Remoção de Stopwords: Todas aquelas pessoas ->  ['todas', 'pessoas']


##### Aplicando a derivação de palavras

In [647]:
def lemmatization_words(token: list[str]) -> list[str]:
    terms = nlp(" ".join(token))

    lemmatized_tokens = [term.lemma_ for term in terms]

    return lemmatized_tokens

In [648]:
print('Aplicação da lematização:', lemmatization_words(['amigo', 'amigos', 'amigas']))

Aplicação da lematização: ['amigo', 'amigo', 'amiga']


###### Algoritmo YAKE

In [649]:
def yake_algorithm(text: str) -> list:
    query = []

    custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_thresold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
    keywords = custom_kw_extractor.extract_keywords(text)

    query = [word for word, score in keywords]

    return query

#### Aplicando as funções nos documentos

In [650]:
def pre_processing(folder: str, document_token: dict, vocabulary: set) -> tuple:
    files = sorted(glob.glob(folder))

    for file in files:
        text = read_file(file)

        text = remove_accents(text)
        text = remove_special_characters(text)
        text = convert_to_lowercase(text)

        token = word_tokenize(text)
        token = remove_stopwords(token)
        token = lemmatization_words(token)

        vocabulary.update(token)

        document_token[os.path.basename(file)] = token

    return document_token, vocabulary

In [651]:
folder = '/content/drive/MyDrive/Information retrieval/data/*'

document_token, vocabulary = pre_processing(folder, {}, set())

In [652]:
print('estrutura de dados [AV - 2](3 - GABARITO):', document_token['estrutura de dados [AV - 2](3 - GABARITO).txt'])

estrutura de dados [AV - 2](3 - GABARITO): ['algoritmos', 'estrutura', 'dar', 'arvor', 'considerar', 'seguinte', 'assertivo', 'sobre', 'arvor', 'binaria', 'buscar', 'variante', 'i', 'arvor', 'Avl', 'altura', 'maxima', 'garantir', 'olog', 'n', 'onde', 'n', 'numero', 'elemento', 'arvorir', 'ii', 'arvor', '23', 'sao', 'variacao', 'arvor', 'binaria', 'buscar', 'permitir', 'dois', 'filho', 'iii', 'arvor', 'b', 'profundidade', 'busco', 'sempre', 'olog', 'n', 'independentemente', 'distribuicao', 'dar', 'iv', 'arvor', 'busca', 'binar', 'balancear', 'avl', 'arvor', 'rubronegra', 'garantir', 'altura', 'arvorir', 'olog', 'n', 'mau', 'caso', 'alternativo', 'todo', 'assertiva', 'estao', 'correta', 'b', 'todo', 'assertiva', 'estao', 'incorreta', 'c', 'apenas', 'assertivo', 'i', 'correto', 'd', 'apenas', 'assertiva', 'ii', 'iii', 'estao', 'correta', 'apenas', 'assertivo', 'iii', 'iv', 'estao', 'correta']


In [653]:
print('Vocabulário:', sorted(vocabulary))

Vocabulário: ['0', '0010010010', '002', '010', '0110110110', '012', '012157913', '020', '0pessoas10', '1', '10', '100', '10000', '1001001001', '101', '11', '1101101101', '12', '12c', '13', '130121579', '13452', '139510716', '14352', '15', '15423', '15432', '17', '19', '1fn', '1n', '2', '20', '20000', '20999', '23', '234', '24', '25', '2a', '2fn', '3', '30', '32', '35', '396', '398', '3fn', '4', '40', '5', '6', '7', '8', '81', '839', '9', '951071613', '975211013', '98', '99', 'A2', 'Abstract', 'Agregacao', 'Assinale', 'Autonomoustransaction', 'Avl', 'Bubble', 'Campos', 'Ccodproduto', 'Classe', 'Classes', 'Cnpj', 'Codproduto', 'Colocao', 'Compilacao', 'Definicao', 'Escola', 'Escritos', 'Estado', 'Estados', 'Estrela', 'Fault', 'Fifo', 'Forint', 'From', 'Geral', 'Grafo', 'Grafos', 'Gravacao', 'Gsignalconnect', 'Gtkcontainer', 'Gtkcontaineradd', 'Gtkinit', 'Gtklabelnew', 'Gtkmainquit', 'Gtkwindgetshowall', 'Heuristicas', 'Insercao', 'Keya2', 'L5', 'Lista', 'Long', 'Loop', 'Main', 'Maxima', 

### Cálculo da frequência

###### Preparação para o cálculo da frequência

In [654]:
class Node:
    def __init__(self, name: str):
        self.name      = name
        self.next      = None

    def __str__(self):
        return f'[Documento: {self.name}]'

class LinkedList:
    def __init__(self):
        self.head = None

    def print_list(self) -> None:
        current_node = self.head

        while current_node:
            print(current_node, end = " -> ")
            current_node = current_node.next

        print("[ ]")

    def get_document_list(self) -> list[Node]:
        array = []
        node = self.head

        while node:
            array.append(node)
            node = node.next

        return array

    def append_document(self, name: str) -> None:
        new_node = Node(name)

        if self.head is None:
            self.head = new_node
            return

        last_node = self.head

        while last_node.next:
            last_node = last_node.next

        last_node.next = new_node

    def postings_size(self) -> int:
        return len(self.get_document_list())

###### Geração da matriz de frequência

In [655]:
def calculate_term_documents(term_documents: dict) -> dict:
    for name, token in document_token.items():
        for term in set(token):
            term_documents[term].append_document(name)

    return term_documents

In [656]:
print('Exemplo de termo-frequência [schedule]:')

term_documents = calculate_term_documents({term: LinkedList() for term in vocabulary})

for document in term_documents['schedule'].get_document_list():
    print(document)

Exemplo de termo-frequência [schedule]:
[Documento: banco de dados [PL-SQL - 3](2).txt]
[Documento: banco de dados [PT - 3](2).txt]


In [657]:
def intersection_tokens(list1, list2):
    return list(set(list1) & set(list2))

In [658]:
def calculate_similarity(term_documents: dict, collection_size: int, query_token: list) -> dict:
    similarity = {}

    for name, token in document_token.items():
        common_terms = intersection_tokens(query_token, token)

        score = 0
        for term in common_terms:
            posting_size = term_documents[term].postings_size()
            score += math.log2((collection_size + 0.5) / (posting_size + 0.5))

        similarity[name] = score

    return similarity

In [659]:
similarity = calculate_similarity(term_documents, len(document_token), document_token['estrutura de dados [AV - 3](2 - GABARITO).txt'])

sorted_similarity = dict(sorted(similarity.items(), key=lambda item: item[1], reverse=True))

table = []
for key, value in sorted_similarity.items():
    table.append([key, f"{value}"])

# print(tabulate(table, headers=['Documento', 'Similaridade']))

In [660]:
def similarity_iterator(relevant_documents: list, query_token: list, collection_size: int) -> dict:
    number_of_relevant_documents = {}

    for term in query_token:
        postings = term_documents[term].get_document_list()
        number_of_relevant_documents[term] = 0

        for post in postings:
            if post.name in relevant_documents:
                number_of_relevant_documents[term] += 1

    answer = {}

    for name, token in document_token.items():
        common_terms = intersection_tokens(query_token, token)

        score = 0
        for term in common_terms:
            ni = term_documents[term].postings_size()
            R  = len(relevant_documents)
            ri = number_of_relevant_documents[term]

            divisor  = (R - ri + 0.5) * (ni - ri + 0.5)
            dividend = (ri + 0.5) * (collection_size - ni - R + ri + 0.5)

            score += math.log2(dividend / divisor)

        answer[name] = score

    return answer

In [661]:
relevant_documents = [
    'estrutura de dados [AV - 2](2 - GABARITO).txt',
    'estrutura de dados [AV - 2](4 - GABARITO).txt',
    # 'estrutura de dados [CA - 2](2 - GABARITO).txt',
    'estrutura de dados [CA - 2](1 - GABARITO).txt',
]

answer = similarity_iterator(relevant_documents, document_token['estrutura de dados [AV - 3](1 - GABARITO).txt'], len(document_token))

sorted_similarity = dict(sorted(answer.items(), key=lambda item: item[1], reverse=True))

table = []
for key, value in sorted_similarity.items():
    table.append([key, f"{value}"])

print(tabulate(table, headers=['Documento', 'Similaridade']))

Documento                                          Similaridade
-----------------------------------------------  --------------
estrutura de dados [AV - 3](1 - GABARITO).txt        35.378
estrutura de dados [AV - 3](2 - GABARITO).txt        18.667
estrutura de dados [MO - 1](1 - GABARITO).txt        15.3938
estrutura de dados [AV - 2](1 - GABARITO).txt        14.8839
estrutura de dados [AV - 2](2 - GABARITO).txt        13.5468
estrutura de dados [AV - 2](4 - GABARITO).txt        13.5468
estrutura de dados [AV - 2](5 - GABARITO).txt        13.4965
estrutura de dados [CA - 2](1 - GABARITO).txt         9.48335
estrutura de dados [CA - 2](2 - GABARITO).txt         9.08115
estrutura de dados [AV - 2](3 - GABARITO).txt         7.26086
banco de dados [MR - 2](10).txt                       5.48702
estrutura de dados [FL - 3](3).txt                    4.64416
estrutura de dados [MO - 1](3).txt                    3.75765
estrutura de dados [MB - 2](1).txt                    3.24776
grafos [CA - 